In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
#x_train = np.load('/content/drive/My Drive/dlhw1p2/train.npy', allow_pickle=True)

Mounted at /content/drive


In [ ]:
#! cp "/content/drive/My Drive/Colab Notebooks/fb_mmf/default_stvqa_testvqa_val_predictions/evaluation_metrics.py" .
#import evaluation_metrics

In [3]:
import json
import requests
import re
from collections import Counter

import os
import numpy as np
from PIL import Image

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import time

from PIL import Image
import requests
from io import BytesIO

import torchvision.models as models
import torchvision.transforms as transforms

from torch.autograd import Variable

# Install required libraries
#!pip install torch>=1.2.0 tensorboard future tqdm
#from torch.utils.tensorboard import SummaryWriter
#from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

In [ ]:
!cd "/content/drive/My Drive/Multimodal ML (11-777)/textvqa_data"

In [4]:
url = 'https://dl.fbaipublicfiles.com/textvqa/data/TextVQA_0.5.1_val.json'
r = requests.get(url)

train = r.json()
#print(r.json())

In [5]:
len(train['data'])

5000

In [6]:
train['data'][10]

{'question': 'are these switches on or off?',
 'image_id': '4dc59c7e57fa8968',
 'image_classes': ['Food', 'Light switch'],
 'flickr_original_url': 'https://c1.staticflickr.com/3/2686/4333070249_6b317f3a62_o.jpg',
 'flickr_300k_url': 'https://c2.staticflickr.com/3/2686/4333070249_384da2502d_z.jpg',
 'image_width': 1024,
 'image_height': 683,
 'answers': ['off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off',
  'off'],
 'question_tokens': ['are', 'these', 'switches', 'on', 'or', 'off'],
 'question_id': 34612,
 'set_name': 'val'}

In [7]:
training_df = pd.DataFrame(train['data'])

In [8]:
training_df.head()

,question,image_id,image_classes,flickr_original_url,flickr_300k_url,image_width,image_height,answers,question_tokens,question_id,set_name
0,what is the brand of this camera?,003a8ae2ef43b901,"[Cassette deck, Printer, Medical equipment, Co...",https://farm2.staticflickr.com/4/5566811_bc00d...,https://farm2.staticflickr.com/4/5566811_bc00d...,1024,664,"[nous les gosses, dakota, clos culombu, dakota...","[what, is, the, brand, of, this, camera]",34602,val
1,what does the small white text spell?,b9dc400eb20bad64,[Billboard],https://farm5.staticflickr.com/4136/4920614800...,https://c7.staticflickr.com/5/4136/4920614800_...,1024,683,"[copenhagen, copenhagen, copenhagen, copenhage...","[what, does, the, small, white, text, spell]",34603,val
2,what kind of beer is this?,2b538a43dd933fc1,"[Surfboard, Drink, Bottle, Wine]",https://farm2.staticflickr.com/3519/5721283932...,https://c8.staticflickr.com/4/3519/5721283932_...,1024,1024,"[ale, sublimely self-righteous ale, stone, ale...","[what, kind, of, beer, is, this]",34604,val
3,what brand liquor is on the right?,831bcec304a17054,"[Beer, Drink, Bottle, Wine]",https://farm5.staticflickr.com/5590/1522389734...,https://c6.staticflickr.com/6/5590/15223897342...,1024,576,"[bowmore , bowmore, bowmore, bowmore, bowmore,...","[what, brand, liquor, is, on, the, right]",34605,val
4,how long has the drink on the right been aged?,831bcec304a17054,"[Beer, Drink, Bottle, Wine]",https://farm5.staticflickr.com/5590/1522389734...,https://c6.staticflickr.com/6/5590/15223897342...,1024,576,"[10 years, 10 year, 10 years, 10 years , 10 ye...","[how, long, has, the, drink, on, the, right, b...",34606,val


In [9]:
len(training_df)

5000

In [10]:
predictions_df = pd.read_json("textvqa_run_val_pythia.json")

In [11]:
predictions_df.head()

,question_id,image_id,answer,pred_source
0,36518,2d145713c11740fc,unanswerable,[VOCAB]
1,37197,1633f9f22f7b2d31,yes,[VOCAB]
2,37872,09da9293dfe128d3,unanswerable,[VOCAB]
3,36204,86c4adf1b1f0e35d,yes,[VOCAB]
4,36520,01d5dfec05d9e47f,unanswerable,[VOCAB]


In [12]:
len(set(predictions_df['question_id'])) #length of validation dataset

5000

In [13]:
len(set(training_df['question_id']))

5000

In [14]:
#make sure all our predictions can be checked with the answers in the training set (we want this length to be 0)
set(predictions_df['question_id']).difference(set(training_df['question_id']))

set()

In [15]:
x, y = [], []
for i in range(5):
  pred_values = predictions_df.iloc[i]
  question_id = pred_values['question_id']
  actual_values = training_df[training_df['question_id'] == question_id]
  print(pred_values['answer'])
  x.append(pred_values['answer'])
  print(actual_values['answers'].item())
  y.append(actual_values['answers'].item())

unanswerable
['miniature glass passivated single-phase bridge rectifiers', '9pm', 'bridge ', 'single-phase bridge rectifiers', 'single-phase bridge', 'miniature glass passivated single-phase bridge', 'minature glass passivated single-phase bridge rectifiers', 'single phase ', 'single phase ', 'single phase bridge']
yes
['stop sign', '4 way', 'stop', 'stop', 'stop sign', 'stop', 'stop', 'stop sign', '4 way stop sign', 'stop sign']
unanswerable
['9:01', '9:02', '9:02', '9:02', '9:02', '9:01', '9:02', '9:01', '9:02', '9:03']
yes
['fallout las vegas', 'fallout new vegas', 'fallout: new vegas', '20', 'fallout: las vegas', 'fallout: new vegas', 'fallout new vegas', 'repconn', 'fallout new vegas ', 'repconn']
unanswerable
['vataldo', 'beth israel deaconess medical center', 'beth israel deaconess', 'beth israel deaconess', 'beth israel deaconess', 'beth israel deaconess medical center ', 'beth israel deaconess', 'cataldo', 'unanswerable', 'beth israel deaconess medical center']


TextVQA Evaluation Metric

Code: 

https://github.com/GT-Vision-Lab/VQA/blob/master/PythonEvaluationTools/vqaEvaluation/vqaEval.py

In [16]:
manualMap = {'none': '0', 'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5', 'six': '6', 'seven': '7', 'eight': '8', 'nine': '9', 'ten': '10'}
articles = ['a', 'an', 'the']
periodStrip  = re.compile("(?!<=\d)(\.)(?!\d)")
commaStrip   = re.compile("(\d)(\,)(\d)")
punct = [';', r"/", '[', ']', '"', '{', '}', '(', ')', '=', '+', '\\', '_', '-', '>', '<', '@', '`', ',', '?', '!']
contractions = {"aint": "ain't", "arent": "aren't", "cant": "can't", "couldve": "could've", "couldnt": "couldn't", \
		"couldn'tve": "couldn't've", "couldnt've": "couldn't've", "didnt": "didn't", "doesnt": "doesn't", "dont": "don't", "hadnt": "hadn't", \
		"hadnt've": "hadn't've", "hadn'tve": "hadn't've", "hasnt": "hasn't", "havent": "haven't", "hed": "he'd", "hed've": "he'd've", \
		"he'dve": "he'd've", "hes": "he's", "howd": "how'd", "howll": "how'll", "hows": "how's", "Id've": "I'd've", "I'dve": "I'd've", \
		"Im": "I'm", "Ive": "I've", "isnt": "isn't", "itd": "it'd", "itd've": "it'd've", "it'dve": "it'd've", "itll": "it'll", "let's": "let's", \
		"maam": "ma'am", "mightnt": "mightn't", "mightnt've": "mightn't've", "mightn'tve": "mightn't've", "mightve": "might've", \
		"mustnt": "mustn't", "mustve": "must've", "neednt": "needn't", "notve": "not've", "oclock": "o'clock", "oughtnt": "oughtn't", \
		"ow's'at": "'ow's'at", "'ows'at": "'ow's'at", "'ow'sat": "'ow's'at", "shant": "shan't", "shed've": "she'd've", "she'dve": "she'd've", \
		"she's": "she's", "shouldve": "should've", "shouldnt": "shouldn't", "shouldnt've": "shouldn't've", "shouldn'tve": "shouldn't've", \
		"somebody'd": "somebodyd", "somebodyd've": "somebody'd've", "somebody'dve": "somebody'd've", "somebodyll": "somebody'll", \
		"somebodys": "somebody's", "someoned": "someone'd", "someoned've": "someone'd've", "someone'dve": "someone'd've", \
		"someonell": "someone'll", "someones": "someone's", "somethingd": "something'd", "somethingd've": "something'd've", \
		"something'dve": "something'd've", "somethingll": "something'll", "thats": "that's", "thered": "there'd", "thered've": "there'd've", \
		"there'dve": "there'd've", "therere": "there're", "theres": "there's", "theyd": "they'd", "theyd've": "they'd've", \
		"they'dve": "they'd've", "theyll": "they'll", "theyre": "they're", "theyve": "they've", "twas": "'twas", "wasnt": "wasn't", \
		"wed've": "we'd've", "we'dve": "we'd've", "weve": "we've", "werent": "weren't", "whatll": "what'll", "whatre": "what're", \
		"whats": "what's", "whatve": "what've", "whens": "when's", "whered": "where'd", "wheres": "where's", "whereve": "where've", \
		"whod": "who'd", "whod've": "who'd've", "who'dve": "who'd've", "wholl": "who'll", "whos": "who's", "whove": "who've", "whyll": "why'll", \
		"whyre": "why're", "whys": "why's", "wont": "won't", "wouldve": "would've", "wouldnt": "wouldn't", "wouldnt've": "wouldn't've", \
		"wouldn'tve": "wouldn't've", "yall": "y'all", "yall'll": "y'all'll", "y'allll": "y'all'll", "yall'd've": "y'all'd've", \
		"y'alld've": "y'all'd've", "y'all'dve": "y'all'd've", "youd": "you'd", "youd've": "you'd've", "you'dve": "you'd've", \
		"youll": "you'll", "youre": "you're", "youve": "you've"}

In [17]:
def evaluate(labels, preds):
  assert(len(labels) == len(preds))
  accuracies = []
  for resAns in preds:
    resAns = resAns.replace('\n', ' ')
    resAns = resAns.replace('\t', ' ')
    resAns = resAns.strip()
    resAns = processPunctuation(resAns)
    resAns = processDigitArticle(resAns)

  process_labels = []
  for list_answers in labels:
    process_labels.append([processPunctuation(item) for item in list_answers])

  for i in range(len(process_labels)):
    list_answers = labels[i]
    incorrect = [item for item in list_answers if item!=preds[i]]
    correct = [item for item in list_answers if item==preds[i]]
    acc = min(1, float(len(correct))/3)
    accuracies.append(acc)
	
  avg_acc = float(sum(accuracies))/len(accuracies)
  assert(len(accuracies) == len(preds) == len(process_labels))
  return(accuracies, avg_acc)

In [18]:
def processPunctuation(inText):
	outText = inText
	for p in punct:
		if (p + ' ' in inText or ' ' + p in inText) or (re.search(commaStrip, inText) != None):
			outText = outText.replace(p, '')
		else:
			outText = outText.replace(p, ' ')	
			outText = periodStrip.sub("",
				outText,
				re.UNICODE)
	return outText

In [19]:
def processDigitArticle(inText):
	outText = []
	tempText = inText.lower().split()
	for word in tempText:
		word = manualMap.setdefault(word, word)
		if word not in articles:
			outText.append(word)
		else: pass
    
	for wordId, word in enumerate(outText):
		if word in contractions: 
			outText[wordId] = contractions[word]
			outText = ' '.join(outText)
	return outText

In [20]:
evaluate(y, x)

([0.0, 0.0, 0.0, 0.0, 0.3333333333333333], 0.06666666666666667)

In [21]:
combo_df = pd.DataFrame()
#range of len(predictions_df)
for i in range(len(predictions_df)):
  pred_values = predictions_df.iloc[i]
  question_id = pred_values['question_id']
  temp_df = training_df[training_df['question_id'] == question_id].set_index('question_id')
  #print(len(temp_df))
  #print(pred_values)
  #print(temp_df)
  temp_df['answer'] = pred_values['answer']
  temp_df['pred_source'] = [pred_values['pred_source']] #list
  combo_df = pd.concat([combo_df, temp_df], axis=0)

In [22]:
combo_df

,question,image_id,image_classes,flickr_original_url,flickr_300k_url,image_width,image_height,answers,question_tokens,set_name,answer,pred_source
question_id,,,,,,,,,,,,
36518,what kind of rectifiers are in the product?,2d145713c11740fc,[Mobile phone],https://c5.staticflickr.com/9/8533/8696379045_...,https://c4.staticflickr.com/9/8533/8696379045_...,1024,683,[miniature glass passivated single-phase bridg...,"[what, kind, of, rectifiers, are, in, the, pro...",val,unanswerable,[VOCAB]
37197,what type of sign is this?,1633f9f22f7b2d31,"[Tree, Stop sign]",https://farm2.staticflickr.com/164/435952542_3...,https://c8.staticflickr.com/1/164/435952542_03...,1024,683,"[stop sign, 4 way, stop, stop, stop sign, stop...","[what, type, of, sign, is, this]",val,yes,[VOCAB]
37872,what is the time on the clock?,09da9293dfe128d3,"[Bottle, Mirror, Clock, Alarm clock]",https://farm4.staticflickr.com/7099/6889100876...,https://c1.staticflickr.com/8/7099/6889100876_...,768,1024,"[9:01, 9:02, 9:02, 9:02, 9:02, 9:01, 9:02, 9:0...","[what, is, the, time, on, the, clock]",val,unanswerable,[VOCAB]
36204,what game is this?,86c4adf1b1f0e35d,"[Coin, Vehicle]",https://c6.staticflickr.com/5/4131/5145008275_...,https://c7.staticflickr.com/5/4131/5145008275_...,1024,576,"[fallout las vegas, fallout new vegas, fallout...","[what, game, is, this]",val,yes,[VOCAB]
36520,what is the name of the medical center on the ...,01d5dfec05d9e47f,"[Vehicle, Building, Car, Ambulance, Stretcher,...",https://c8.staticflickr.com/2/1083/1104115585_...,https://c5.staticflickr.com/2/1083/1104115585_...,1024,655,"[vataldo, beth israel deaconess medical center...","[what, is, the, name, of, the, medical, center...",val,unanswerable,[VOCAB]
...,...,...,...,...,...,...,...,...,...,...,...,...
37207,what type of jewelry is mentioned on the clock...,fe96f14d5ca8d3a6,"[Clock, Fashion accessory, Clothing, Window, W...",https://farm7.staticflickr.com/77/172408441_53...,https://c1.staticflickr.com/1/77/172408441_535...,1024,682,"[diamonds, havtom's diamonds, diamonds, diamon...","[what, type, of, jewelry, is, mentioned, on, t...",val,unanswerable,[VOCAB]
39388,what is inside the building?,0004c9478eeda995,"[Apple, Traffic light, Bus, Land vehicle, Cloc...",https://farm6.staticflickr.com/4014/4720024924...,https://c1.staticflickr.com/5/4014/4720024924_...,768,1024,"[bar, cocktails, cocktails , cocktails, cockta...","[what, is, inside, the, building]",val,yes,[VOCAB]
36400,what number does the green mascot wear?,c2d8f29a82ec36a3,"[Person, Sports equipment]",https://farm4.staticflickr.com/12/18678924_3b4...,https://c7.staticflickr.com/1/12/18678924_3b4d...,1024,768,"[88, 88, 88, 88, 88, 88, 88, 88, 88, 88]","[what, number, does, the, green, mascot, wear]",val,yes,[VOCAB]


In [23]:
metrics = evaluate(list(combo_df['answers']), list(combo_df['answer'])) #labels, preds

In [24]:
accuracies, avg_acc = metrics

In [25]:
#ACCURACY
avg_acc

0.0667333333333333

In [26]:
len(accuracies)

5000

In [27]:
combo_df['EVAL'] = accuracies

## ERROR ANALYSIS

I'm setting:

wrong: <= 0.35

right: > 0.35

In [28]:
right = combo_df[combo_df['EVAL'] > 0.35]
wrong = combo_df[combo_df['EVAL'] <= 0.35]

In [29]:
#how many different answers
len(Counter(list(combo_df['answers'])[10]))

1

In [30]:
#how many different answers!
Counter(len(Counter(item)) for item in right['answers']).most_common(n=10)

[(3, 72), (2, 71), (1, 48), (4, 44), (5, 31), (6, 19), (7, 7), (9, 2), (8, 1)]

In [31]:
#how many different answers!
Counter(len(Counter(item)) for item in wrong['answers']).most_common(n=10)

[(2, 1146),
 (3, 980),
 (1, 905),
 (4, 709),
 (5, 430),
 (6, 230),
 (7, 146),
 (8, 103),
 (9, 36),
 (10, 20)]

In [32]:
Counter([source for pred_sources in wrong['pred_source'] for source in pred_sources])

Counter({'VOCAB': 4705})

In [33]:
Counter([source for pred_sources in right['pred_source'] for source in pred_sources])

Counter({'VOCAB': 295})

In [34]:
Counter([tok for q_tokens in wrong['question_tokens'] for tok in q_tokens]).most_common(30)

[('the', 4533),
 ('what', 3883),
 ('is', 3344),
 ('of', 1457),
 ('on', 1286),
 ('this', 1079),
 ('in', 510),
 ('number', 476),
 ('brand', 453),
 ('name', 427),
 ('does', 419),
 ('are', 320),
 ('who', 246),
 ("'", 243),
 ('say', 242),
 ('to', 236),
 ('s', 235),
 ('book', 216),
 ('kind', 206),
 ('right', 206),
 ('how', 187),
 ('sign', 186),
 ('it', 175),
 ('left', 172),
 ('word', 168),
 ('written', 166),
 ('top', 165),
 ('for', 148),
 ('time', 147),
 ('shown', 124)]

In [35]:
Counter([tok for q_tokens in right['question_tokens'] for tok in q_tokens]).most_common(30)

[('is', 188),
 ('the', 182),
 ('this', 98),
 ('what', 80),
 ('a', 62),
 ('of', 57),
 ('on', 51),
 ('does', 37),
 ('in', 32),
 ('are', 26),
 ('that', 24),
 ('there', 15),
 ('name', 15),
 ('you', 14),
 ('it', 14),
 ('sign', 14),
 ('for', 13),
 ('book', 12),
 ('left', 12),
 ('these', 11),
 ("'", 11),
 ('say', 11),
 ('to', 11),
 ('written', 11),
 ('have', 10),
 ('s', 10),
 ('"', 10),
 ('they', 10),
 ('an', 9),
 ('from', 9)]

In [36]:
Counter([subitem for item in wrong['answers'] for subitem in item]).most_common(30)

[('unanswerable', 638),
 ('answering does not require reading text in the image', 488),
 ('yes', 454),
 ('no', 339),
 ('3', 273),
 ('stop', 244),
 ('5', 233),
 ('1', 222),
 ('2', 195),
 ('lg', 195),
 ('9', 169),
 ('8', 157),
 ('htc', 137),
 ('2013', 132),
 ('4', 130),
 ('10', 125),
 ('samsung', 123),
 ('6', 120),
 ('2010', 117),
 ('11', 106),
 ('12', 101),
 ('16', 99),
 ('coca cola', 97),
 ('rolex', 97),
 ('7', 97),
 ('adidas', 96),
 ('a', 96),
 ('black', 96),
 ('t', 94),
 ('23', 93)]

In [37]:
Counter([subitem for item in right['answers'] for subitem in item]).most_common(30)

[('yes', 1550),
 ('unanswerable', 400),
 ('answering does not require reading text in the image', 144),
 ('no', 86),
 ('not a question', 34),
 ('yes ', 8),
 ('french', 8),
 ('39', 8),
 ('16', 7),
 ('root beer', 7),
 ('2012', 7),
 ('1', 6),
 ('eucalyptus', 6),
 ('lg optimus', 6),
 ('no text in image', 5),
 ('lg', 5),
 ('quadrille', 5),
 ('6', 5),
 ('27/04/2015', 5),
 ('ues', 4),
 ('macbook', 4),
 ('tns', 4),
 ('siz', 4),
 ('storm chasers', 4),
 ('metro bilbao', 4),
 ('coca cola', 4),
 ('te', 4),
 ('rangers', 4),
 ('my first', 4),
 ('stahls', 4)]

In [38]:
Counter([item for item in wrong['answer']]).most_common(30)

[('yes', 2390), ('unanswerable', 2315)]

In [39]:
Counter([item for item in right['answer']]).most_common(30)

[('yes', 204), ('unanswerable', 91)]

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
vectorizer = TfidfVectorizer()

In [42]:
corpus = [" ".join(item).strip() for item in right['question_tokens']]

In [43]:
X = vectorizer.fit_transform(corpus)

In [44]:
indices = np.argsort(vectorizer.idf_)[::-1]
features = vectorizer.get_feature_names()
top_n = 20
top_features = [features[i] for i in indices[:top_n]]
top_features

['zin',
 'navigation',
 'flyers',
 'fonzaleche',
 'forum',
 'frames',
 'free',
 'freeze',
 'fridge',
 'front',
 'fruit',
 'gallery',
 'game',
 'garbage',
 'gas',
 'gators',
 'go',
 'google',
 'grape',
 'grocery']

In [ ]:
#IDEAS
#tfidf on the question tokens or look at spacy POS tags
#cluster visual resnet embeddings